# 한 채널의 모든 동영상 이름, 재생시간, 날짜 크롤링하기

각자 수정이 필요한 부분에 **강조** 표시를 해두겠습니다!  
꼭 코드를 돌리면서 맥락을 파악해보세요.

In [1]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
# 필요한 함수들을 불러줍니다!
import re
import time
from pandas import DataFrame
import pandas as pd
import numpy as np

**파일 경로는 각자 환경에 맞게 수정하기**

In [4]:
# youtube_unique_0717.csv 파일을 불러와주세요!
df = pd.read_csv('C:\\Users\\김지희\\DSL\\dataset\\youtube_unique_0717.csv', lineterminator='\n')

In [5]:
# 채널 아이디의 고유한 값을 channel_unique 변수로 저장해주세요!
channel_unique = df['channelId'].unique()

In [6]:
# 채널에 대해서 일단 긁어오는 거니까 용량을 많이 차지하는 df는 없애줍시다!!!
df = 0

In [7]:
# 아래의 함수는 스크롤을 한 번만 내려주는 스크롤입니다.
# 스크롤을 한 번 내리면 한 채널당 90개까지의 영상을 크롤링해올 수 있습니다.
# 저희의 목표는 한 채널의 모든 동영상의 정보를 다 가져오는 것이기에, 스크롤을 내려야만 그 정보들을 가져올 수 있습니다!
# https://heytech.tistory.com/325 참고하였습니다.
# 이 셀은 그냥 실행시켜주시면 됩니다! 맥락만 이해해주세요.


def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        count = 0
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = 3
            # 페이지 최하단까지 스크롤
            browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            count += 1
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
            
            
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if count == 1 or new_page_height == last_page_height:
                # 한 번 스크롤한 경우 혹은, 스크롤을 더 내리지 못하는 경우
                #print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

**각자 환경에 맞게 수정해주세요**

In [8]:
# 크롬드라이버 경로 설정 (새로운 창이 열릴 겁니다!)
browser = webdriver.Chrome(r'C:\\Users\\김지희\\Downloads\\chromedriver.exe')

C:\ProgramData\Anaconda3\envs\Crawling\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [9]:
from bs4 import BeautifulSoup

반복문 돌아가기 전에 예시로 간단히 돌려보겠습니다.

In [10]:
channelid = channel_unique[2]
url = 'https://www.youtube.com/channel/%s/videos'%channelid
browser.get(url)
    
scroll()
print('%d번째 채널 스크롤 완료'%2)
    
page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
# 뷰티풀숩으로 창에 존재하는 정보들을 모두 가져오기

2번째 채널 스크롤 완료


In [11]:
#뷰티풀숩으로 동영상들이 있는 부분의 내용만을 다 끌어오기
all_videos = soup.find_all(id='dismissible')
    
#재생시간 예시
video_time_list = []
for video in all_videos:
    # all_videos는 페이지에 존재하는 동영상 모두.
    # video는 이 동영상 하나하나를 반복문으로 접근하게 됩니다.
    
    video_time = video.find('span',{'class' : 'style-scope ytd-thumbnail-overlay-time-status-renderer'})
    # 재생시간 데려오기 (이때 video_time 자체를 출력하면 뷰티풀숩 뭐시기가 뜹니다)
    video_time_list.append(video_time.text.strip())
    # video_time에서 text 속성이 바로 재생시간 문자열 그 자체입니다.
    # 빈칸이 있는 경우 strip하고, 리스트에 추가해줍니다.
    # 이를 페이지에 존재하는 동영상 모두에 대해서 실행합니다.
        
# 원 데이터와 merge할 video_id를 찾는 과정입니다.
video_link_list = []
for video in all_videos:
    title = video.find(id='video-title')
    # title 변수 안에는 aria-label, href, title 속성값이 존재합니다.
    # 이 중에서 비디오 자체의 링크를 가지고 있는 속성 : 'href'
    if len(title['href'].strip())>0:
        if 'shorts/' in title['href']:
            video_link_list.append(title['href'][8:])
            # 전체 링크 중에서 video_id만 데려오기 위한 인덱싱
        else:
            video_link_list.append(title['href'][9:])
            # 전체 링크 중에서 video_id만 데려오기 위한 인덱싱

In [13]:
print(len(video_link_list))

90


In [14]:
# 왜 재생시간을 구할 때
# video.find('span',{'class' : 'style-scope ytd-thumbnail-overlay-time-status-renderer'})를 사용했는지,
# 왜 video_id를 찾을 때
# title = video.find(id='video-title')을 하고 다시 ['href']를 씌웠는지
# 출력한 결과를 보면 구조를 확인하면서 깨달으실 수 있을 거예요!
print(all_videos)

[<div class="style-scope ytd-grid-video-renderer" id="dismissible"><ytd-thumbnail class="style-scope ytd-grid-video-renderer" use-hovered-property=""><!--css-build:shady--><a aria-hidden="true" class="yt-simple-endpoint inline-block style-scope ytd-thumbnail" href="/watch?v=6KUJQxKXkhY" id="thumbnail" rel="null" tabindex="-1">
<yt-image alt="" class="style-scope ytd-thumbnail" disable-upgrade="" ftl-eligible="" hidden="" notify-on-loaded="" notify-on-unloaded="">
</yt-image>
<yt-img-shadow class="style-scope ytd-thumbnail no-transition" ftl-eligible="" loaded="" style="background-color: transparent;"><!--css-build:shady--><img alt="" class="style-scope yt-img-shadow" draggable="false" id="img" src="https://i.ytimg.com/vi/6KUJQxKXkhY/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&amp;rs=AOn4CLBuoxhBdijRXQUDKwaeMd-o4b2LEA" width="210"/></yt-img-shadow>
<div class="style-scope ytd-thumbnail" id="overlays"><ytd-thumbnail-overlay-time-status-renderer class="style-scope y

In [15]:
when_list = []
for video in all_videos:
    when = video.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})
    # 재생시간이랑 코드가 비슷한데, 다른 것은 find_all 입니다. 
    # 첫번째 태그가 재생시간이고 두번째 태그가 업로드날 입니다.
    when = when[1].text
    when_list.append(when.strip())

In [16]:
when_list

['1일 전',
 '4일 전',
 '6일 전',
 '13일 전',
 '2주 전',
 '2주 전',
 '2주 전',
 '3주 전',
 '3주 전',
 '3주 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전']

In [17]:
full = DataFrame(columns = ['channelId','video_time','video_id','when', 'title'])

t = 0
for i in range(500,1000):
    try:
        channelid = channel_unique[i]
        url = 'https://www.youtube.com/channel/%s/videos'%channelid
        browser.get(url)
    
        scroll()
        print('%d번째 채널 스크롤 완료'%i)
    
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
    
        all_videos = soup.find_all(id='dismissible')
    
        title_list = []
        for video in all_videos:
            title = video.find(id='video-title')
            if len(title.text.strip())>0:
                title_list.append(title.text)
        # 공백을 제거하고 글자수가 0보다 크면 append  
    
        video_time_list = []
        for video in all_videos:
            video_time = video.find('span',{'class' : 'style-scope ytd-thumbnail-overlay-time-status-renderer'})
            video_time_list.append(video_time.text.strip())
        
        video_link_list = []
        for video in all_videos:
            title = video.find(id='video-title')
            if len(title['href'].strip())>0:
                if 'shorts/' in title['href']:
                    video_link_list.append(title['href'][8:])
                else:
                    video_link_list.append(title['href'][9:])
                    #원래 데이터와의 videoid concat을 위한 작업
                
        when_list = []
        for video in all_videos:
            when = video.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})
            # 재생시간이랑 코드가 비슷한데, 다른 것은 find_all 입니다. 
            # 첫번째 태그가 재생시간이고 두번째 태그가 업로드날 입니다.
            when = when[1].text
            when_list.append(when.strip())
            #언제 업로드 되었는지 : n일전으로 뜸
        
        if (len(video_time_list) == len(video_link_list)):
            youtube = pd.DataFrame({'channelId':channelid,'title':title_list, 'video_time':video_time_list,'video_id':video_link_list,
                                    'when':when_list})
            full = pd.concat([full, youtube],ignore_index = True)
        else:
            t = t+1
            print('%d번째 채널은 이상함'%i)
            
    except:
        t=t+1
        print('%d번째 채널은 이상함'%i)

500번째 채널 스크롤 완료
501번째 채널 스크롤 완료
502번째 채널 스크롤 완료
503번째 채널 스크롤 완료
504번째 채널 스크롤 완료
505번째 채널 스크롤 완료
506번째 채널 스크롤 완료
507번째 채널 스크롤 완료
508번째 채널 스크롤 완료
509번째 채널 스크롤 완료
510번째 채널 스크롤 완료
511번째 채널 스크롤 완료
512번째 채널 스크롤 완료
513번째 채널 스크롤 완료
514번째 채널 스크롤 완료
515번째 채널 스크롤 완료
516번째 채널 스크롤 완료
517번째 채널 스크롤 완료
518번째 채널 스크롤 완료
519번째 채널 스크롤 완료
520번째 채널 스크롤 완료
521번째 채널 스크롤 완료
522번째 채널 스크롤 완료
522번째 채널은 이상함
523번째 채널 스크롤 완료
524번째 채널 스크롤 완료
525번째 채널 스크롤 완료
526번째 채널 스크롤 완료
527번째 채널 스크롤 완료
528번째 채널 스크롤 완료
529번째 채널 스크롤 완료
530번째 채널 스크롤 완료
531번째 채널 스크롤 완료
532번째 채널 스크롤 완료
533번째 채널 스크롤 완료
534번째 채널 스크롤 완료
535번째 채널 스크롤 완료
536번째 채널 스크롤 완료
537번째 채널 스크롤 완료
538번째 채널 스크롤 완료
539번째 채널 스크롤 완료
539번째 채널은 이상함
540번째 채널 스크롤 완료
540번째 채널은 이상함
541번째 채널 스크롤 완료
541번째 채널은 이상함
542번째 채널 스크롤 완료
542번째 채널은 이상함
543번째 채널 스크롤 완료
543번째 채널은 이상함
544번째 채널 스크롤 완료
544번째 채널은 이상함
545번째 채널 스크롤 완료
545번째 채널은 이상함
546번째 채널 스크롤 완료
546번째 채널은 이상함
547번째 채널 스크롤 완료
547번째 채널은 이상함
548번째 채널 스크롤 완료
548번째 채널은 이상함
549번째 채널 스크롤 완료
549번째 채널은 이상함
550번째 채널 스크롤 완료
550번째 채널은 이상함
55

950번째 채널 스크롤 완료
951번째 채널 스크롤 완료
952번째 채널 스크롤 완료
953번째 채널 스크롤 완료
954번째 채널 스크롤 완료
955번째 채널 스크롤 완료
955번째 채널은 이상함
956번째 채널 스크롤 완료
957번째 채널 스크롤 완료
958번째 채널 스크롤 완료
959번째 채널 스크롤 완료
959번째 채널은 이상함
960번째 채널 스크롤 완료
960번째 채널은 이상함
961번째 채널 스크롤 완료
962번째 채널 스크롤 완료
963번째 채널 스크롤 완료
964번째 채널 스크롤 완료
965번째 채널 스크롤 완료
966번째 채널 스크롤 완료
967번째 채널 스크롤 완료
968번째 채널 스크롤 완료
969번째 채널 스크롤 완료
970번째 채널 스크롤 완료
971번째 채널 스크롤 완료
972번째 채널 스크롤 완료
973번째 채널 스크롤 완료
974번째 채널 스크롤 완료
975번째 채널 스크롤 완료
976번째 채널 스크롤 완료
977번째 채널 스크롤 완료
978번째 채널 스크롤 완료
978번째 채널은 이상함
979번째 채널 스크롤 완료
980번째 채널 스크롤 완료
981번째 채널 스크롤 완료
982번째 채널 스크롤 완료
983번째 채널 스크롤 완료
984번째 채널 스크롤 완료
985번째 채널 스크롤 완료
986번째 채널 스크롤 완료
987번째 채널 스크롤 완료
988번째 채널 스크롤 완료
989번째 채널 스크롤 완료
990번째 채널 스크롤 완료
991번째 채널 스크롤 완료
992번째 채널 스크롤 완료
993번째 채널 스크롤 완료
994번째 채널 스크롤 완료
995번째 채널 스크롤 완료
996번째 채널 스크롤 완료
997번째 채널 스크롤 완료
998번째 채널 스크롤 완료
999번째 채널 스크롤 완료


In [18]:
# 드디어 끝났습니다! 출력하셔서 확인해보세요~
full

,channelId,video_time,video_id,when,title
0,UC9pdRyCVY9foIQid-YDjm4Q,9:02,Yoyj_YYc_f4,2개월 전,[PL] 35R 토트넘 vs 레스터 HL ｜ FIFA22로 미리보기 (ft.캐스터 ...
1,UC9pdRyCVY9foIQid-YDjm4Q,5:55,B6vk5wSIBoU,2개월 전,K4에서 K2로 3경기 만에 콜업돼서 주전 먹은 01년생 센터백ㄷㄷ
2,UC9pdRyCVY9foIQid-YDjm4Q,5:20,vABMh2E_Htk,3개월 전,"조현우 공백 완벽히 지운 백업 골키퍼, 조수혁의 놀라운 실력과 멘털리티"
3,UC9pdRyCVY9foIQid-YDjm4Q,10:52,dlxQT2fFA-s,3개월 전,[PL] 34R 브렌트포드 vs 토트넘 HL ｜ FIFA22로 미리보기 (ft.캐스...
4,UC9pdRyCVY9foIQid-YDjm4Q,35:55,xY-PNi9wBLg,3개월 전,"수원 삼성, 이병근 감독 부임 기자회견 풀버전!… ""수원의 부활 이룰 것"""
...,...,...,...,...,...
34074,UCfsxn3UrWFIl9VSgN3XUBPA,3:08,Y8-3faay8P4,11개월 전,영상툰) 유치원생 동생이 이걸 먹으려 해요! [ 썰툰ㅣ사연툰 ]
34075,UCfsxn3UrWFIl9VSgN3XUBPA,12:41,a9ZYC8J4Glk,11개월 전,영상툰) 손자 때문에 할머니가 죽을 뻔 했어요! 이상한 가족썰 모음 [ 썰툰ㅣ사이다툰 ]
34076,UCfsxn3UrWFIl9VSgN3XUBPA,4:02,sDpeuBstiZ0,11개월 전,영상툰) 친구가 촉법소년이 될 뻔했어요! [ 썰툰ㅣ사연툰 ]
34077,UCfsxn3UrWFIl9VSgN3XUBPA,3:10,-C4cpcr2B1A,11개월 전,영상툰) 유명 맛집 김치에 XX가 들어있어요! [ 썰툰ㅣ애니메이션 ]


In [19]:
# 90개를 가져오기 위해 스크롤을 내린 것이었는데요,
# 채널별로 딱 영상이 두 개만 있는 경우도 있네요.
full['channelId'].value_counts()

UC9pdRyCVY9foIQid-YDjm4Q    90
UCQVn712WYOFCXDHSrs0UcdA    90
UCRqYpgUV0xPDWignRgTzp6g    90
UC_U34UlsS9Mp5ZgvsEA7irw    90
UCoUDrzyCl1IwU602xdTsM-g    90
                            ..
UCQIJ1GlCJu2_tIrAD86XqmQ    14
UC1K0SJqmbK5yePofxFhMygg    10
UC8MwgENB-GXGGknn6AhzRsA     6
UCTJK06VpQffu-VG_yZDfVTg     3
UCi85Tjhbb3gNLBUv_j_Tlpw     1
Name: channelId, Length: 419, dtype: int64

In [20]:
# 데려와놓고 저장 안하면 완전 꽝이죠! 저장도 꼭 해주어요.
# 본인에게 할당된 range에서의 첫번호를 적어 저장합니다. channel_첫번호
# 합쳐서 한꺼번에 전처리하는 것이 편할 것 같아, 일단 이렇게 저장해주세요.

full.to_csv('C:\\Users\\김지희\\DSL\\dataset\\channel_500.csv', index = False)

In [21]:
# 잘 불러와지는지까지 확인하면 끝!
df = pd.read_csv('C:\\Users\\김지희\\DSL\\dataset\\channel_500.csv')

In [22]:
df.head()

,channelId,video_time,video_id,when,title
0,UC9pdRyCVY9foIQid-YDjm4Q,9:02,Yoyj_YYc_f4,2개월 전,[PL] 35R 토트넘 vs 레스터 HL ｜ FIFA22로 미리보기 (ft.캐스터 ...
1,UC9pdRyCVY9foIQid-YDjm4Q,5:55,B6vk5wSIBoU,2개월 전,K4에서 K2로 3경기 만에 콜업돼서 주전 먹은 01년생 센터백ㄷㄷ
2,UC9pdRyCVY9foIQid-YDjm4Q,5:20,vABMh2E_Htk,3개월 전,"조현우 공백 완벽히 지운 백업 골키퍼, 조수혁의 놀라운 실력과 멘털리티"
3,UC9pdRyCVY9foIQid-YDjm4Q,10:52,dlxQT2fFA-s,3개월 전,[PL] 34R 브렌트포드 vs 토트넘 HL ｜ FIFA22로 미리보기 (ft.캐스...
4,UC9pdRyCVY9foIQid-YDjm4Q,35:55,xY-PNi9wBLg,3개월 전,"수원 삼성, 이병근 감독 부임 기자회견 풀버전!… ""수원의 부활 이룰 것"""


In [23]:
#드디어 창을 나가줍니다.
browser.quit()